In [ ]:
from transformers import AutoProcessor, AutoModelForCausalLM
import torch

from huggingface_hub import hf_hub_download
from PIL import Image

#### Load dataset

In [ ]:
import boto3
import config
from dataloader import SNV3Dataset, collate_fn
from torch.utils.data import Dataset, DataLoader
import time

session = boto3.Session(
    aws_access_key_id=config.aws_access_key_id,
    aws_secret_access_key=config.aws_secret_access_key,
)
s3 = session.resource('s3')
train_ds = SNV3Dataset(split='train',
                  bucket='soccernet-230',
                  vocab_path='vocab_files/train_vocab.pyi',
                  num_clips=2, window_size=3,
                  include_vid=True,
                     local_video_path="/raid/videos")
train_dataloader = DataLoader(train_ds, collate_fn=collate_fn, batch_size=2, num_workers=10)

In [ ]:
batch = next(iter(train_dataloader))
for k,v in batch.items():
    print(k,v.shape)

In [ ]:
#ggg

#### Define Model

In [ ]:
from transformers import AutoProcessor, AutoModelForCausalLM
import torch

from huggingface_hub import hf_hub_download
from PIL import Image

# video captioning
processor = AutoProcessor.from_pretrained("microsoft/git-base-vatex")
model = AutoModelForCausalLM.from_pretrained("microsoft/git-base-vatex")

#### Dummy forward pass

In [ ]:
# for x in batch['caption']:
#     print(len(x))
#     print(train_ds.text_processor.detokenize(x.tolist()))

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

In [ ]:
frames = batch['clip'].permute(0, 1, 4, 2, 3).float()
input_text = [train_ds.text_processor.detokenize(x.tolist()) for x in batch['caption']]
#input_ids, mask, pixel_values = processor(text=input_text, return_tensors="pt", padding=True).to(device)
#print(pixel_values.shape)
#generated_ids = model.generate(pixel_values=pixel_values, max_length=50)
#frames = processor(images=frames, return_tensors="pt").pixel_values

input_ids = processor(text=input_text, add_special_tokens=False, padding='max_length').input_ids
mask = processor(text=input_text, add_special_tokens=False, padding='max_length').attention_mask
#print(input_ids)
outputs = model(input_ids=torch.tensor(input_ids).to(device),
                attention_mask=torch.tensor(mask).to(device),
                pixel_values=frames.to(device),
                labels=torch.tensor(input_ids).to(device))
outputs.loss

#### Train the model

In [ ]:


# import torch

# optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)

# model.train()

# for epoch in range(50):
#   print("Epoch:", epoch)
#   for idx, batch in enumerate(train_dataloader):
        
        
#     frames = batch['clip'].permute(0, 1, 4, 2, 3).float()
#     input_text = [train_ds.text_processor.detokenize(x.tolist()) for x in batch['caption']]
#     #input_ids, mask, pixel_values = processor(text=input_text, return_tensors="pt", padding=True).to(device)
#     #print(pixel_values.shape)
#     #generated_ids = model.generate(pixel_values=pixel_values, max_length=50)
#     #frames = processor(images=frames, return_tensors="pt").pixel_values

#     input_ids = processor(text=input_text, add_special_tokens=False, padding='max_length').input_ids
#     mask = processor(text=input_text, add_special_tokens=False, padding='max_length').attention_mask
#     #print(input_ids)
#     outputs = model(input_ids=torch.tensor(input_ids).to(device),
#                     #attention_mask=torch.tensor(mask).to(device),
#                     pixel_values=frames.to(device),
#                     labels=torch.tensor(input_ids).to(device))
# #     input_ids = batch.pop("input_ids").to(device)
# #     pixel_values = batch.pop("pixel_values").to(device)

# #     outputs = model(input_ids=input_ids,
# #                     pixel_values=pixel_values,
# #                     labels=input_ids)
    
#     loss = outputs.loss

#     print("Loss:", loss.item())

#     loss.backward()

#     optimizer.step()
#     optimizer.zero_grad()